In [1]:
from pymongo import MongoClient
import pandas as pd
import geopandas as gpd
import json
import src.functions as fun

conn = MongoClient("localhost:27017")
db = conn.get_database("ironhack")
c = db.get_collection("spain_companies")

import folium
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster

#### These are the coordinates of our new offices:

In [44]:
office_lat = 40.427308
office_lon = -3.689512

In [79]:
map_1 = folium.Map(location= [office_lat, office_lon], zoom_start=14)
map_1

#### We add our new office to a list to plot them with Folium:

In [4]:
places_list = list()

In [5]:
places_list.append({"name": "Our new office", "place_type": "office", "latitude": office_lat, "longitude": office_lon})

In [6]:
places_list

[{'name': 'Our new office',
  'place_type': 'office',
  'latitude': 40.427308,
  'longitude': -3.689512}]

#### Then we proceed with the criterias of our database (1M companies and videogames companies near by):

In [7]:
n = "Virgin Play"
coord = {"type": "Point", "coordinates": [office_lat, office_lon]}

# First query for videogames companies near:
query1 = {"$and": 
         [{"location": 
           {"$near": 
            {"$geometry": coord, 
            "$minDistance": 1, 
            "$maxDistance": 1000
          }}},
           {"category_code": 
            "games_video"},
          {"name": 
           {"$ne": n}}
                 ]}
final_query1 = c.find(query1)
results1 = list(final_query1)

# Second query for companies with more than 1M dollares raised:
query2 = {"$and": 
         [{"location": 
           {"$near": 
            {"$geometry": coord, 
            "$minDistance": 1, 
            "$maxDistance": 1000
          }}},
           {"raised_amount": 
            {"$gte": 1000000}},
          {"name": 
           {"$ne": n}}
                 ]}
final_query2 = c.find(query2)
results2 = list(final_query2)

In [8]:
fun.mongo_places(results1, "Videogame company", places_list)

In [9]:
fun.mongo_places(results2, "1M company", places_list)

In [10]:
places_list

[{'name': 'Our new office',
  'place_type': 'office',
  'latitude': 40.427308,
  'longitude': -3.689512},
 {'name': 'eRepublik',
  'place_type': 'Videogame company',
  'latitude': 40.43094,
  'longitude': -3.6952896},
 {'name': 'eRepublik',
  'place_type': '1M company',
  'latitude': 40.43094,
  'longitude': -3.6952896},
 {'name': 'Busuu',
  'place_type': '1M company',
  'latitude': 40.4225487,
  'longitude': -3.6830316}]

#### Now we get again every places with Google API to visualize them with coordinates:

#### First, let's get those Starbucks:

In [11]:
response1 = fun.get_places(office_lat, office_lon, 1000, keyword = "starbucks")

In [12]:
fun.google_places(response1, "Starbucks", places_list)

#### Airport:

In [15]:
response2 = fun.get_places(office_lat, office_lon, 1000, type_ = "airport")

In [16]:
fun.google_places(response2, "Airport", places_list)

#### Basketball stadiums:

In [18]:
response3 = fun.get_places(office_lat, office_lon, 10000, type_ = "stadium", keyword = "basketball")

In [19]:
fun.google_places(response3, "Basketball stadium", places_list)

#### Vegan restaurants:

In [20]:
response4 = fun.get_places(office_lat, office_lon, 1000, type_ = "restaurant", keyword = "vegan")

In [21]:
fun.google_places(response4, "Vegan restaurant", places_list)

#### Schools:

In [22]:
response5 = fun.get_places(office_lat, office_lon, 1000, type_ = "school")

In [23]:
fun.google_places(response5, "School", places_list)

#### Night clubs:

In [24]:
response6 = fun.get_places(office_lat, office_lon, 1000, type_ = "night_club")

In [25]:
fun.google_places(response6, "Nightclub", places_list)

#### Dog Hairdressers:

In [26]:
response7 = fun.get_places(office_lat, office_lon, 1000, type_ = "pet_store", keyword = "hairdresser")

In [27]:
fun.google_places(response7, "Dog hairdresser", places_list)

In [29]:
len(places_list)

87

In [32]:
pd_places = pd.DataFrame(places_list)

In [73]:
pd_places["place_type"].unique()

array(['office', 'Videogame company', '1M company', 'Starbucks',
       'Airport', 'Basketball stadium', 'Vegan restaurant', 'School',
       'Nightclub', 'Dog hairdresser'], dtype=object)

In [56]:
pd_places.to_json("data/final_places.json", orient="records")

#### Let's make a map with all the sites of interest:

In [80]:
for i, row in pd_places.iterrows():
    place = {
        "location": [row["latitude"], row["longitude"]],
        "tooltip": row["name"]
    }
    
    if row["place_type"] == "office":
        icon = Icon (
            color="blue",
            prefix="fa",
            icon="briefcase",
            icon_color="black"
        )
    elif row["place_type"] == "Videogame company":
        icon = Icon (
            color="red",
            prefix="fa",
            icon="gamepad",
            icon_color="black"
        )
    elif row["place_type"] == "1M company":
        icon = Icon (
            color="green",
            prefix="fa",
            icon="briefcase",
            icon_color="black"
        )
    elif row["place_type"] == "Starbucks":
        icon = Icon (
            color="green",
            prefix="fa",
            icon="coffee",
            icon_color="white"
        )    
    elif row["place_type"] == "Airport":
        icon = Icon (
            color="black",
            prefix="fa",
            icon="plane",
            icon_color="white"
        )
    elif row["place_type"] == "Vegan restaurant":
        icon = Icon (
            color="green",
            prefix="fa",
            icon="cutlery",
            icon_color="white"
        )
    elif row["place_type"] == "School":
        icon = Icon (
            color="darkred",
            prefix="fa",
            icon="graduation-cap",
            icon_color="black"
        )
    elif row["place_type"] == "Nightclub":
        icon = Icon (
            color="purple",
            prefix="fa",
            icon="glass",
            icon_color="black"
        )
    elif row["place_type"] == "Dog hairdresser":
        icon = Icon (
            color="beige",
            prefix="fa",
            icon="paw",
            icon_color="black"
        )
    else:
        icon = Icon (
            color="red",
            prefix="fa",
            icon="futbol-o",
            icon_color="white"
        )        
    Marker(**place, icon=icon).add_to(map_1)
map_1